In [1]:
import numpy as np
import joblib
import glob
from tqdm.contrib.concurrent import process_map
import gc
from tqdm.notebook import tqdm

In [ ]:
# data_base = glob.glob("/workspace/data/conc_data/*.npz")
# len(data_base)
# max_len = 0

# file_count = len(data_base)
# bar = tqdm(desc="proccessing files", total=file_count)
# tot = 0

# for filename in data_base:
#     try: 
#         bar.update(1)
#         tot += 1
#         data = np.load(filename, allow_pickle=True)
#         coef = data['arr_0']
#         if coef.shape[1] > max_len:
#             max_len = coef.shape[1]
#     except: print("problem at: {}".format(filename))

# print(max_len)

In [2]:
def padd(coef, required_len):
    pad_size = required_len - coef.shape[1]
    result = np.zeros((coef.shape[0], required_len))
    result[:coef.shape[0],:coef.shape[1]] = coef
    return result

In [3]:
def process2(filename):
    required_len = 141000
    data = np.load(filename, allow_pickle=True)
    coef, label = data['arr_0'], data['arr_2']
    res = padd(coef, required_len)
    
    #save
    filename = filename.replace("conc_data", "Padded") # -> change dir
    filename = filename.replace(".wav.npz", "") 
    np.savez_compressed(filename+"_padded"+'.npz', res, label)
   


In [4]:
a=process_map(process2, glob.glob("/workspace/data/conc_data/*.npz"), max_workers=8, smoothing=0.0, chunksize=4)

  0%|          | 0/5252 [00:00<?, ?it/s]

In [2]:
padded = glob.glob("/workspace/data/Padded/*.npz")
padded[0]

'/workspace/data/Padded/03-01-07-01-02-01-23_padded.npz'

In [ ]:
X = []
y = []
for sample in padded:
    data = np.load(sample, allow_pickle=True)
    X.append(data['arr_0'])
    y.append(data['arr_1'])
    